# **[ 멘토멘티 5주차 스터디 ]** 

# 0. Data

In [ ]:
import numpy as np
from keras.utils import to_categorical
from keras.datasets import cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 
# 32*32 픽셀 크기의 컬러 이미지 60000개로 구성된 data
# 이미지는 10개의 라벨로 분류되어 있음. 

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
n = 10

x_train = x_train.astype('float32') / 255.0 # input을 -1~1 사이 범위로 조정하기 위함
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, n) # 10개 라벨을 원핫으로 펼침
y_test = to_categorical(y_test, n)

In [ ]:
x_train[0][0]

array([[0.23137255, 0.24313726, 0.24705882],
       [0.16862746, 0.18039216, 0.1764706 ],
       [0.19607843, 0.1882353 , 0.16862746],
       [0.26666668, 0.21176471, 0.16470589],
       [0.38431373, 0.28627452, 0.20392157],
       [0.46666667, 0.35686275, 0.24705882],
       [0.54509807, 0.41960785, 0.29411766],
       [0.5686275 , 0.43137255, 0.3137255 ],
       [0.58431375, 0.45882353, 0.34901962],
       [0.58431375, 0.47058824, 0.3647059 ],
       [0.5137255 , 0.40392157, 0.3019608 ],
       [0.49019608, 0.3882353 , 0.29803923],
       [0.5568628 , 0.4509804 , 0.35686275],
       [0.5647059 , 0.4392157 , 0.3372549 ],
       [0.5372549 , 0.4117647 , 0.30980393],
       [0.5058824 , 0.38039216, 0.2784314 ],
       [0.5372549 , 0.41568628, 0.30980393],
       [0.5254902 , 0.41568628, 0.29803923],
       [0.4862745 , 0.38039216, 0.2509804 ],
       [0.54509807, 0.44313726, 0.30588236],
       [0.54509807, 0.4392157 , 0.29411766],
       [0.52156866, 0.4117647 , 0.27058825],
       [0.

In [ ]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)


x_train, x_test 는 각각 **4차원 텐서**임. 
- 첫번째 : 이미지의 인덱스(6만개중에 5만개가 train)
- 두번째, 세번쨰 : 이미지의 높이와 너비(32*32 픽셀크기)
- 마지막 : 컬러 채널(RGB이므로 3)

In [ ]:
# ex
x_train[20, 12, 13]   # 20번째 이미지의 (12, 13) 위치에 있는 RGB 채널 값

array([0.40392157, 0.39607844, 0.38431373], dtype=float32)

# 1. 정말 간단한 신경망

In [ ]:
from keras.layers import Input, Flatten, Dense # 3종류의 layer
from keras.models import Model

input_layer = Input( shape = (32, 32, 3)) # 배치 크기는 지정하지 않음. 

x = Flatten()(input_layer) # dense를 위해 펼친다 > 벡터의 길이 = 32*32*3 = 3072일 것임. 

x = Dense(units = 200, activation = 'relu')(x)
x = Dense(units = 150, activation = 'relu')(x)

output_layer = Dense(units = 10, activation = 'softmax')(x) # sortmax 

model = Model(input_layer, output_layer) # model = input, output으로 정의됨. 
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 200)               614600    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               30150     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1510      
Total params: 646,260
Trainable params: 646,260
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# model compile
from keras.optimizers import Adam
opt = Adam(lr = 0.0005)
model.compile(loss = 'categorical_crossentropy', optimizer = opt)
metrics = ['accuracy']

In [ ]:
# fit
model.fit(x_train, y_train, batch_size = 32, epochs = 10, shuffle = True)

Epoch 1/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.8486
Epoch 2/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.6637
Epoch 3/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.5842
Epoch 4/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.5297
Epoch 5/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.4918
Epoch 6/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.4624
Epoch 7/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.4336
Epoch 8/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.4106
Epoch 9/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.3838
Epoch 10/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.3699


In [ ]:
from keras.layers import Input, Flatten, Dense # 3종류의 layer
from keras.models import Model
from keras.layers import BatchNormalization

input_layer = Input( shape = (32, 32, 3)) # 배치 크기는 지정하지 않음. 

x = Flatten()(input_layer) # dense를 위해 펼친다 > 벡터의 길이 = 32*32*3 = 3072일 것임. 

x = Dense(units = 200, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dense(units = 150, activation = 'relu')(x)
x = BatchNormalization()(x)

output_layer = Dense(units = 10, activation = 'softmax')(x) # sortmax 

model = Model(input_layer, output_layer) # model = input, output으로 정의됨. 
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               614600    
_________________________________________________________________
batch_normalization (BatchNo (None, 200)               800       
_________________________________________________________________
dense_5 (Dense)              (None, 150)               30150     
_________________________________________________________________
batch_normalization_1 (Batch (None, 150)               600       
_________________________________________________________________
dense_6 (Dense)              (None, 10)               

# 2. Convolution

In [ ]:
from keras.layers import Conv2D
input_layer = Input(shape = (32,32,3))

conv1 = Conv2D(filters = 10, kernel_size = (4,4), strides = 2, padding = 'same')(input_layer)
conv2 = Conv2D(filters = 20, kernel_size = (3,3), strides = 2, padding = 'same')(conv1)

f = Flatten()(conv2)
output_layer = Dense(units = 10, activation='softmax')(f)

model = Model(input_layer, output_layer)
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 10)        490       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 20)          1820      
_________________________________________________________________
flatten_4 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                12810     
Total params: 15,120
Trainable params: 15,120
Non-trainable params: 0
_________________________________________________________________
